In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import AE
from models import DNN
import numpy as np
import pandas as pd

In [2]:
from scipy.stats import pearsonr

# Parameters

In [3]:
# Define parameters
epochs = 500 #200,500,1000
dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1

# Import data

In [4]:
data_r=pd.read_csv('data/GDSCexpression.csv',index_col=0)
label_r=pd.read_csv('data/GDSClabel.csv',index_col=0)

In [5]:
label_r=label_r.fillna(1)

# Your is gene-cell, mine is cell-gene

In [6]:
len(set(data_r.index))

727

In [7]:
data = data_r
label = label_r.iloc[:,10]
#scaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
scaler = preprocessing.MinMaxScaler()


data = scaler.fit_transform(data)

In [8]:
print(np.std(data))
print(np.mean(data))

0.22300475047574178
0.3343586031066485


In [9]:
data.shape

(789, 11833)

In [10]:
label_r.shape

(789, 139)

# Split test train

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)

In [12]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(789, 11833)
(789,)
(631, 11833) (631,)
(158, 11833) (158,)


In [13]:
print(X_train.max())
print(X_train.min())

1.0000000000000009
0.0


In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [15]:
trainData = torch.FloatTensor(X_train).to(device)
testData = torch.FloatTensor(X_test).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
allData = torch.FloatTensor(data).to(device)

# construct TensorDataset
train_dataset = TensorDataset(trainData, trainData)
test_dataset = TensorDataset(testData, testData)
all_dataset = TensorDataset(allData, allData)

trainDataLoader1 = DataLoader(dataset=train_dataset, batch_size=200, shuffle=False)
trainDataLoaderall = DataLoader(dataset=all_dataset, batch_size=200, shuffle=True)

# The model

In [16]:
# Initialization

#autoencoder = AE(dim=train_dataset.shape[1]).to(device)
autoencoder = AE(dim_au_in = X_train.shape[1],dim_au_out=dim_au_out).to(device)
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
#loss1
loss_func = nn.MSELoss().to(device)
#loss2
#loss_func = nn.BCELoss()
loss_train = np.zeros((epochs, 1))

In [18]:
# Train autoencoder

for epoch in range(epochs):
    # 不需要label，所以用一个占位符"_"代替
    for batchidx, (x, _) in enumerate(trainDataLoaderall):
        x.requires_grad_(True)
        # encode and decode 
        decoded, encoded = autoencoder(x)
        # compute loss
        print(encoded.shape, decoded.shape)
        loss = loss_func(decoded, x)      
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
    loss_train[epoch,0] = loss.item() /X_train.shape[0] 
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item()))

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0001, Training loss=0.13187715
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0002, Training loss=0.07502985
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0003, Training loss=0.06970185
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0004, Training loss=0.05143594
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0037, Training loss=0.02807791
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0038, Training loss=0.02761473
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0039, Training loss=0.02773271
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0040, Training loss=0.02704118
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0074, Training loss=0.02218001
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0075, Training loss=0.02172032
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0076, Training loss=0.02111897
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0077, Training loss=0.02180113
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0110, Training loss=0.01840116
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0111, Training loss=0.01826477
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0112, Training loss=0.01798570
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0113, Training loss=0.01841871
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0146, Training loss=0.01658798
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0147, Training loss=0.01611970
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0148, Training loss=0.01640501
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0149, Training loss=0.01602597
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

...

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0329, Training loss=0.01004618
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0330, Training loss=0.01017370
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0331, Training loss=0.00996321
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0332, Training loss=0.01015133
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0365, Training loss=0.00947556
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0366, Training loss=0.00946540
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0367, Training loss=0.00950130
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0368, Training loss=0.00927369
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

Epoch: 0400, Training loss=0.00885742
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0401, Training loss=0.00878543
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0402, Training loss=0.00894576
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0403, Training loss=0.00895816
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0404, Training loss=0.00907086
torch.Size([200, 512]) torch.Size([200, 11

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0437, Training loss=0.00842276
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0438, Training loss=0.00843683
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0439, Training loss=0.00886588
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0440, Training loss=0.00876702
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0473, Training loss=0.00813214
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0474, Training loss=0.00799580
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0475, Training loss=0.00834007
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0476, Training loss=0.00819607
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

In [ ]:
torch.save(autoencoder.state_dict(), 'saved/models/AE_GDSCold.pkl')

In [ ]:
# extract features
_, encodedTrainData = autoencoder(trainData)
featureTensor = encodedTrainData.double()
feature = featureTensor.detach().cpu().numpy()

In [ ]:
print(feature.shape)

In [ ]:
feature

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(feature, Y_train.values)

In [ ]:
_,testFeature = autoencoder(testData)
lasso = clf.predict(testFeature.detach().cpu().numpy())

In [ ]:
r2_score(lasso,Y_test)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature)

In [ ]:
tsne_results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(tsne_results[:, 0], tsne_results[:, 1], label="label")
plt.legend()
plt.savefig("saved/figures/tsne_ae_gdsc.png")

In [ ]:
EPOCH = 500

In [ ]:
# Load data
# data type conversion
B_feature = torch.FloatTensor(feature).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
b_data = TensorDataset(B_feature, y)
trainDataLoader2 = DataLoader(dataset=b_data, batch_size=200, shuffle=False)

In [ ]:
b_data.tensors

In [ ]:
b_data

In [ ]:
dim_dnn_in

In [ ]:
# Initialization DNN model

predictor = DNN(dim_dnn_in, dim_dnn_out).to(device)
optimizer = optim.Adam(predictor.parameters(), lr=1e-3,betas=(0.9,0.99))
#loss1-softmax
loss_func = nn.MSELoss().to(device)
#loss2-sigmoid
#loss_func = nn.BCELoss().to(device)
#loss3-sigmoid
#loss_func = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=True)
#criterion = torch.nn.BCELoss(size_average=True) # Defined loss function
#optimizer = optim.Adm(model.parameters(), lr=0.01) # Defined optimizer
loss_train = np.zeros((epochs, 1))

In [ ]:
# train model
for epoch in range(EPOCH):
    print('Epoch: ',epoch)
    for step,(batch_x,batch_y) in enumerate(trainDataLoader2):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        # predict label
        output = predictor(b_x)
        # b_y=F.sigmoid(b_y) 
        
        #print
        #print(output)
        #print(b_y)
        # compute loss
        loss = loss_func(output,b_y)
        #loss = criterion(output, b_y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item())) 

# Save model
torch.save(predictor.state_dict(), 'saved/models/DNN_GDSC.pkl')

In [ ]:
# Get tesing feature
_,testFeature = autoencoder(testData)

In [ ]:
testpredict = predictor(testFeature)

In [ ]:
testpredict

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
r2_score(testpredict.detach().cpu().numpy(),Y_test)

In [ ]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)